#  LeNet-5
    first arch, 2 CNN and 3 full layers

In [15]:
import torch
import torch.nn as nn

In [31]:
class LeNet5(nn.Module):
    def __init__(self,in_channels = 1,num_classes=10):
        super(LeNet5,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels,6,kernel_size = 5)
        self.relu1 = nn.ReLU()
        self.avg1 = nn.AdaptiveAvgPool2d(6)
        self.conv2 = nn.Conv2d(6,16,kernel_size=5)
        self.relu2 = nn.ReLU()
        self.avg2 = nn.AdaptiveAvgPool2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,num_classes)
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.avg1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.avg2(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

      
lenet5 = LeNet5() 
print(lenet5)

LeNet5(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (avg1): AdaptiveAvgPool2d(output_size=6)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (avg2): AdaptiveAvgPool2d(output_size=16)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


# AlexNNet
    used GPU and ReLU first

In [64]:
class AlexNNet(nn.Module):
    def __init__(self,in_channels=3,num_classes=1000):
        super(AlexNNet,self).__init__()
        # 11 3 5 4 6 3
        self.conv1 = nn.Conv2d(in_channels, 96, kernel_size=11, stride=4)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()

        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()

        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()

        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2)

        self.drop = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(256 * 6 * 6,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,num_classes)
    
    def forward(self,x):

        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)

        x = self.conv3(x)
        x = self.relu3(x)

        x = self.conv4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.relu5(x)

        x = self.maxpool3(x)

        x = x.view(x.size(0), -1)
        x = self.drop(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

BATCH_SIZE = 5
y = torch.randn(BATCH_SIZE, 3, 227, 227)
alex_net = AlexNNet()
print(alex_net(y).shape)

torch.Size([5, 1000])


# DenseNet
    something about growth rate
    broke my brain

In [100]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate):
        super(DenseLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1)
        self.batch_norm = nn.BatchNorm2d(growth_rate)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = F.relu(out)
        out = torch.cat((x, out), 1)
        return out

class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, num_layers):
        super(DenseBlock, self).__init__()
        self.layers = nn.ModuleList([DenseLayer(in_channels + i*growth_rate, growth_rate) for i in range(num_layers)])
        
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.batch_norm(out)
        out = F.relu(out)
        out = self.avg_pool(out)
        return out

class DenseNet(nn.Module):
    def __init__(self, num_classes, growth_rate=32, block_config=(6, 12, 24, 16), compression=0.5):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate
        

        self.conv1 = nn.Conv2d(3, 2*growth_rate, kernel_size=7, stride=2, padding=3)
        self.batch_norm1 = nn.BatchNorm2d(2*growth_rate)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        

        in_channels = 2*growth_rate
        for i, num_layers in enumerate(block_config):
            block = DenseBlock(in_channels, growth_rate, num_layers)
            setattr(self, f'denseblock{i+1}', block)
            in_channels += num_layers * growth_rate
            if i != len(block_config) - 1:
                trans = TransitionLayer(in_channels, int(in_channels * compression))
                setattr(self, f'transition{i+1}', trans)
                in_channels = int(in_channels * compression)
        

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(in_channels, num_classes)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.batch_norm1(out)
        out = F.relu(out)
        out = self.pool1(out)
        
        for i in range(1, 5):
            out = getattr(self, f'denseblock{i}')(out)
            if i != 4:
                out = getattr(self, f'transition{i}')(out)
        
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out


def DenseNet121():
    BATCH_SIZE = 5
    y = torch.randn(BATCH_SIZE, 3, 227, 227)
    model = DenseNet(num_classes=10, growth_rate=32, block_config=(6, 12, 24, 16), compression=0.5)
    print(model(y).shape)

DenseNet121()

torch.Size([5, 10])
